# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

### Importando bibliotecas

In [95]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [96]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import os
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

### Conectando ao MySQL

In [97]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

In [98]:
'''statement = text('SELECT * FROM pnad_covid_view LIMIT 5')
response = mysql.execute(statement)
for row in response:
        print(row)'''

"statement = text('SELECT * FROM pnad_covid_view LIMIT 5')\nresponse = mysql.execute(statement)\nfor row in response:\n        print(row)"

### Criando sessão Spark

In [99]:
os.environ['SPARK_HOME'] = '/opt/spark/'
findspark.init()

In [100]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [101]:
spark

### Importando dados

In [102]:
'''res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))
res = res.fetchall()'''

"res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))\nres = res.fetchall()"

In [103]:
'''df = pd.DataFrame(res)'''

'df = pd.DataFrame(res)'

In [104]:
'''df.to_csv('dados/dados_exportados/2023-10-25_pnad_covid_view.csv', index=False)'''

"df.to_csv('dados/dados_exportados/2023-10-25_pnad_covid_view.csv', index=False)"

### Análise exploratória de dados econômicos

In [105]:
df = spark.read.csv('dados/dados_exportados/2023-10-25_pnad_covid_view.csv', sep = ',', inferSchema = True, header = True)


In [130]:
df.show()

+----------+-----------------+------------------+-----+------+--------+--------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+--------------------+--------------------+-------------+-----------------------+-----------+---------------------------+--------------------+-----------------+---------------+
|      data|               uf|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_internacao|questao_interna

In [106]:
df.printSchema()

root
 |-- data: date (nullable = true)
 |-- uf: string (nullable = true)
 |-- situacao_domicilio: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cor_raca: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- questao_estabelecimento_saude: string (nullable = true)
 |-- questao_permaneceu_casa: string (nullable = true)
 |-- questao_remedio_conta_propria: string (nullable = true)
 |-- questao_remedio_orientacao_medica: string (nullable = true)
 |-- questao_hospital_SUS: string (nullable = true)
 |-- questao_hospital_privado: string (nullable = true)
 |-- questao_internacao: string (nullable = true)
 |-- questao_internacao_ajuda_respirar: string (nullable = true)
 |-- questao_motivo_afastamento: string (nullable = true)
 |-- questao_tempo_afastado_trab: string (nullable = true)
 |-- questao_tipo_trabalho_realizado: string (nullable = true)
 |-- faixa_rendimento: string (nullable = true)
 |-- rendimento_aposentado

In [107]:
df.columns

['data',
 'uf',
 'situacao_domicilio',
 'idade',
 'sexo',
 'cor_raca',
 'escolaridade',
 'questao_estabelecimento_saude',
 'questao_permaneceu_casa',
 'questao_remedio_conta_propria',
 'questao_remedio_orientacao_medica',
 'questao_hospital_SUS',
 'questao_hospital_privado',
 'questao_internacao',
 'questao_internacao_ajuda_respirar',
 'questao_motivo_afastamento',
 'questao_tempo_afastado_trab',
 'questao_tipo_trabalho_realizado',
 'faixa_rendimento',
 'rendimento_aposentadoria_pensao',
 'rendimento_bolsa_familia',
 'rendimento_beneficios',
 'auxlio_emergencia_covid',
 'seguro_desemprego',
 'tipo_domicilio',
 'valor_pago_domicilio',
 'sintoma_covid',
 'descricao_sintoma_covid',
 'teste_covid',
 'descricao_fator_risco_covid',
 'tipo_teste',
 'fator_risco_covid',
 'resultado_teste']

### Dicionários

In [129]:
estados = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE', 
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI', 
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

In [128]:
regioes = {
'AC': 'Norte',
'AL': 'Nordeste',
'AP': 'Norte',
'AM': 'Norte',
'BA': 'Nordeste',
'CE': 'Nordeste',
'DF': 'Centro-Oeste',
'ES': 'Sudeste',
'GO': 'Centro-Oeste',
'MA': 'Nordeste',
'MT': 'Centro-Oeste',
'MS': 'Centro-Oeste', 
'MG': 'Sudeste',
'PA': 'Norte',
'PB': 'Nordeste',
'PR': 'Sul',
'PE': 'Nordeste',
'PI': 'Nordeste',
'RJ': 'Sudeste',
'RN': 'Nordeste',
'RS': 'Sul',
'RO': 'Norte',
'RR': 'Norte',
'SC': 'Sul',
'SP': 'Sudeste',
'SE': 'Nordeste',
'TO': 'Norte'
}

#### Adicionando colunas

In [137]:
df = df.withColumn('sigla', df.uf).replace(to_replace=estados, subset=['sigla']) #adicionando sigla dos estados
df = df.withColumn('regiao', df.sigla).replace(to_replace=regioes, subset=['regiao']) #adicionando regioes dos estados
df.show()

+----------+-----------------+------------------+-----+------+--------+--------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+--------------------+--------------------+-------------+-----------------------+-----------+---------------------------+--------------------+-----------------+---------------+-----+--------+
|      data|               uf|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_internacao|

In [145]:
df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

#### Contagio por estado

In [171]:
cotagio_por_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

In [173]:
fig = px.bar(
    cotagio_por_estado.qtd_infectados,
    color = cotagio_por_estado.uf
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Contagio por região

In [175]:
cotagio_por_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) as qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

In [177]:
fig = px.bar(
    cotagio_por_regiao.qtd_infectados,
    color = cotagio_por_regiao.regiao
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Taxa de contagio por estado

In [170]:
df_taxa_de_contagio = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

df2 = spark.sql(
    '''
        SELECT uf, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

df_taxa_de_contagio['taxa_de_contagio_por_mil_habitantes'] = ((df_taxa_de_contagio['qtd_infectados'] / df2['qtd_testes_validos']) * 1000).round().astype(int)

In [ ]:
fig = px.bar(
    df_taxa_de_contagio.taxa_de_contagio_por_mil_habitantes,
    color = df_taxa_de_contagio.uf
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Taxa de contagio por idade nos estados

#### Taxa de contagio por região

In [159]:
df_taxa_de_contagio_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

df2 = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

df_taxa_de_contagio_regiao['taxa_de_contagio_por_mil_habitantes'] = ((df_taxa_de_contagio_regiao['qtd_infectados'] / df2['qtd_testes_validos']) * 1000).round().astype(int)

In [169]:
fig = px.bar(
    df_taxa_de_contagio_regiao.taxa_de_contagio_por_mil_habitantes,
    color = df_taxa_de_contagio_regiao.regiao
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Taxa de contagio por idade nas regioes

#### Taxa de desemprego de infectados por estados

#### Renda média de infectados por estado

#### Métodos de testagem por estado

#### Que áreas profissionais foram mais afetadas por afastamento no período?

In [198]:
df = df.groupBy('questao_tipo_trabalho_realizado').count().orderBy('questao_tipo_trabalho_realizado').show(truncate=False)

+-----------------------------------------------------------------------------------------------+-----+
|questao_tipo_trabalho_realizado                                                                |count|
+-----------------------------------------------------------------------------------------------+-----+
|Agricultor, criador de animais, pescador, silvicultor e jardineiro                             |45720|
|Artesão, costureiro e sapateiro                                                                |9022 |
|Artista, religioso (padre, pastor etc.)                                                        |1070 |
|Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)                                 |12106|
|Auxiliar de escritório, escriturário                                                           |17282|
|Auxiliar de produção, de carga e descarga;                                                     |11568|
|Balconista, vendedor de loja                                   